## ANALYSIS OF THE DATASET



In [3]:
import pandas as pd
import numpy as np
import os
from pathlib import Path


In [4]:
# Get the parent directory of the current working directory and append 'dataset'
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'dataset'))

# Construct the paths for 'test.csv' and 'train.csv'
test_file = os.path.abspath(os.path.join(parent_dir, 'test.csv'))
train_file = os.path.abspath(os.path.join(parent_dir, 'train.csv'))

# Print the paths of the different files
print("Test file path:", test_file)
print("Train file path:", train_file)


Test file path: c:\Users\delos\Documents\GitHub\Dream_Team_2024\dataset\test.csv
Train file path: c:\Users\delos\Documents\GitHub\Dream_Team_2024\dataset\train.csv


In [5]:
df_test = pd.read_csv(test_file, sep=',', low_memory=False)
df_train = pd.read_csv(train_file, sep=',', low_memory=False)

In [ ]:
dataframes = [(df_train, 'df_train'), (df_test, 'df_test')]

# Iterate through each DataFrame and save column info to a text file
for df, df_name in dataframes:
    with open(f'{df_name}_column_info.txt', 'w') as file:
        # Write the header with the column name and non-null count
        file.write("Column Name | Non-null Count | Proportion of Non-null\n")
        file.write("-" * 50 + "\n")
        
        # Iterate through each column and write the count and proportion of non-null values
        total_rows = len(df)  # Total number of rows in the DataFrame
        for column in df.columns:
            non_null_count = df[column].notnull().sum()  # Count of non-null values
            non_null_proportion = non_null_count / total_rows  # Proportion of non-null values
            file.write(f"{column} | {non_null_count} | {non_null_proportion:.4f}\n")  # Format the proportion to 4 decimal places

    print(f"Data has been written to '{df_name}_column_info.txt'")

TypeError: TextIOWrapper.write() takes exactly one argument (2 given)